In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
! pip install pandas-profiling
import pandas_profiling
train.profile_report()

In [3]:
train = train.drop(columns=['2ndFlrSF','3SsnPorch','Alley','BsmtFinSF1','BsmtFinSF2','EnclosedPorch','Fence','FireplaceQu','LowQualFinSF','MasVnrArea','MiscFeature','MiscVal','PoolQC','ScreenPorch','WoodDeckSF'])
test = test.drop(columns=['2ndFlrSF','3SsnPorch','Alley','BsmtFinSF1','BsmtFinSF2','EnclosedPorch','Fence','FireplaceQu','LowQualFinSF','MasVnrArea','MiscFeature','MiscVal','PoolQC','ScreenPorch','WoodDeckSF'])
[train.shape, test.shape]

[(1460, 66), (1459, 65)]

In [4]:
train = train.fillna(0)
test = test.fillna(0)

In [5]:
pd.set_option('display.max_rows', train.shape[0]+1)
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinType2      object
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
HeatingQC         object
CentralAir        object
Electrical        object
1stFlrSF           int64


In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values=np.nan, strategy='mean')
si.fit(train)

In [7]:
[train.shape, test.shape]

[(1460, 66), (1459, 65)]

In [8]:
train_cat = train.select_dtypes('object')
train_cat = train_cat.astype('str')
train_num = train.select_dtypes('int64','float64')

test_cat = test.select_dtypes('object')
test_cat = test_cat.astype('str')
test_num = test.select_dtypes('int64','float64')

[train_num.shape, test_num.shape]

[(1460, 26), (1459, 19)]

In [ ]:
! pip install sklearn
from sklearn.decomposition import PCA
n_components = 2
pca = PCA(n_components)
pca_reduced = pca.fit_transform(train_num)

In [ ]:
import seaborn as sns

# Append the principle components for each entry to the dataframe
for i in range(0, n_components):
    train_cat['PC' + str(i + 1)] = pca_reduced[:, i]
 
display(train_cat.head())
 
# Do a scree plot
ind = np.arange(0, n_components)
(fig, ax) = plt.subplots(figsize=(8, 6))
sns.pointplot(x=ind, y=pca.explained_variance_ratio_)
ax.set_title('Scree plot')
ax.set_xticks(ind)
ax.set_xticklabels(ind)
ax.set_xlabel('Component Number')
ax.set_ylabel('Explained Variance')
plt.show()
 
g = sns.lmplot(
    'PC1',
    'PC2',
    data=train_cat,
    fit_reg=False,
    scatter=True,
    size=7,
    )
 
plt.show()
 
# Plot a variable factor map for the first two dimensions.
(fig, ax) = plt.subplots(figsize=(12, 12))
for i in range(0, len(pca.components_)):
    ax.arrow(0, 0,  # Start the arrow at the origin
             pca.components_[0, i], pca.components_[1, i],  # 0 and 1 correspond to dimension 1 and 2
             head_width=0.1,head_length=0.1)
    plt.text(pca.components_[0, i] + 0.05, pca.components_[1, i] + 0.05, train_cat.columns.values[i])
 
an = np.linspace(0, 2 * np.pi, 100)  # Add a unit circle for scale
plt.plot(np.cos(an), np.sin(an))
plt.axis('equal')
ax.set_title('Variable factor map')
plt.show()

In [ ]:
pca.components_

In [9]:
test_cat.shape

(1459, 38)

In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
train_cat = ohe.fit_transform(train_cat)
test_cat = ohe.transform(test_cat)
[train.shape, test.shape]

[(1460, 66), (1459, 65)]

In [12]:
feat = ohe.get_feature_names()

array(['x0_0.0', 'x0_1.0', 'x1_0.0', 'x1_1.0', 'x2_0.0', 'x2_1.0',
       'x3_0.0', 'x3_1.0', 'x4_0.0', 'x4_1.0', 'x5_0.0', 'x5_1.0',
       'x6_0.0', 'x6_1.0', 'x7_0.0', 'x7_1.0', 'x8_0.0', 'x8_1.0',
       'x9_0.0', 'x9_1.0', 'x10_0.0', 'x10_1.0', 'x11_0.0', 'x11_1.0',
       'x12_0.0', 'x12_1.0', 'x13_0.0', 'x13_1.0', 'x14_0.0', 'x14_1.0',
       'x15_0.0', 'x15_1.0', 'x16_0.0', 'x16_1.0', 'x17_0.0', 'x17_1.0',
       'x18_0.0', 'x18_1.0', 'x19_0.0', 'x19_1.0', 'x20_0.0', 'x20_1.0',
       'x21_0.0', 'x21_1.0', 'x22_0.0', 'x22_1.0', 'x23_0.0', 'x23_1.0',
       'x24_0.0', 'x24_1.0', 'x25_0.0', 'x25_1.0', 'x26_0.0', 'x26_1.0',
       'x27_0.0', 'x27_1.0', 'x28_0.0', 'x28_1.0', 'x29_0.0', 'x29_1.0',
       'x30_0.0', 'x30_1.0', 'x31_0.0', 'x31_1.0', 'x32_0.0', 'x32_1.0',
       'x33_0.0', 'x33_1.0', 'x34_0.0', 'x34_1.0', 'x35_0.0', 'x35_1.0',
       'x36_0.0', 'x36_1.0', 'x37_0.0', 'x37_1.0', 'x38_0.0', 'x38_1.0',
       'x39_0.0', 'x39_1.0', 'x40_0.0', 'x40_1.0', 'x41_0.0', 'x41_1.0'

In [14]:
train_cat = pd.DataFrame(data=train_cat, columns=feat) 
test_cat = pd.DataFrame(data=test_cat, columns=feat) 
[train.shape, test.shape]

[(1460, 66), (1459, 65)]

In [15]:
train_num.reset_index(drop=True, inplace=True)
train_cat.reset_index(drop=True, inplace=True)
#train = pd.concat([train_num],[train_cat])
train = pd.concat([train_num, train_cat], axis=1, sort=False)

test_num.reset_index(drop=True, inplace=True)
test_cat.reset_index(drop=True, inplace=True)
#test = pd.concat([test_num],[test_cat])
test = pd.concat([test_num, test_cat], axis=1, sort=False)

[train_num.shape, test_num.shape, train_cat.shape, test_cat.shape, train.shape, test.shape]

[(1460, 26), (1459, 19), (1460, 490), (1459, 490), (1460, 516), (1459, 509)]

In [16]:
from sklearn.model_selection import train_test_split
val, train = train_test_split(train, random_state=42, test_size=0.8)

In [17]:
X_train = train.drop(columns=['SalePrice'])
y_train = train['SalePrice']
X_val = val.drop(columns=['SalePrice'])
y_val = val['SalePrice']
[X_train.shape, y_train.shape, X_val.shape, y_val.shape]

[(1168, 515), (1168,), (292, 515), (292,)]

In [ ]:
! rd /S /Q logs

In [18]:
import tensorflow as tf
import tensorboard
%load_ext tensorboard

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model = Sequential()
model.add(Dense(288, input_shape=(269,)))
model.add(Dense(288, kernel_initializer='normal', activation='relu'))
model.add(Dense(144, kernel_initializer='normal', activation='relu'))
model.add(Dense(144, kernel_initializer='normal', activation='relu'))
model.add(Dense(18, kernel_initializer='normal', activation='relu'))
model.add(Dense(9, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='relu'))
model.compile(loss='mean_squared_logarithmic_error', optimizer=sgd)

Using TensorFlow backend.


In [20]:
from keras.callbacks import EarlyStopping
import datetime
callbacks = [EarlyStopping(monitor='loss', patience=10), tf.keras.callbacks.TensorBoard("logs_base_dir")]

In [22]:
model.fit(X_train.iloc[:,1:], y_train, batch_size=32, epochs=300, callbacks=callbacks)

ValueError: Error when checking input: expected dense_1_input to have shape (269,) but got array with shape (514,)

In [ ]:
%tensorboard --logdir logs_base_dir

In [ ]:
model.evaluate(X_val.iloc[:,1:], y_val)

In [ ]:
test

In [ ]:
results = model.predict(test.iloc[:,1:])
results = pd.DataFrame(data=[test['Id'], results], columns=['Id','SalePrice'])
results

In [ ]:
! pip install kaggle


In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
print(pca.explained_variance_)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train)

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, shuffle=False, train_size=0.8)